In [ ]:
!wget -nc https://lazyprogrammer.me/course_files/sp500_closefull.csv

File ‘sp500_closefull.csv’ already there; not retrieving.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('sp500_closefull.csv', index_col=0, parse_dates=True)

In [ ]:
df.head()

,CSCO,UAL,TROW,ISRG,PRGO,TPR,DVN,MRO,BA,VRTX,GILD,NLSN,EQIX,TIF,MDT,V,QRVO,A,MO,SWKS,MCHP,CDNS,WLTW,CHTR,EIX,BBY,WBA,HCA,AJG,DTE,C,T,CF,DISH,MGM,XL,HUM,CBOE,CFG,WU,...,DLTR,HAS,WMT,NTAP,KIM,BAX,LMT,KEY,UNM,BMY,PSA,WYNN,RHI,EFX,NUE,PKG,NBL,CTSH,SWK,MU,TRV,L,AEP,CI,SNI,JNJ,WM,DOV,FTI,AGN,M,CRM,PGR,WAT,BWA,LRCX,NWL,UAA,BLK,PPL
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-04,24.690001,12.80,54.400002,102.923332,40.349998,36.310001,76.570000,19.153616,56.180000,44.240002,21.650000,NaN,109.559998,43.990002,43.900002,22.035000,NaN,22.389128,19.790001,14.82,29.180000,6.12,70.913910,NaN,34.770000,40.189999,37.299999,NaN,22.330000,43.540001,34.000000,28.580000,18.368000,21.180000,9.73,NaN,45.680000,NaN,NaN,19.090000,...,16.056667,32.540001,54.230000,34.189999,13.66,31.662140,76.849998,5.94,19.940001,25.629999,80.129997,63.959999,27.120001,31.469999,47.790001,23.389999,36.610001,23.400000,52.709999,10.85,49.810001,37.180000,34.939999,36.369999,NaN,64.680000,34.160000,28.395418,29.615000,40.290001,17.059999,18.705000,18.030001,61.630001,16.889999,39.880001,15.20,3.51000,238.580002,30.242558
2010-01-05,24.580000,13.91,55.009998,102.459999,38.790001,36.750000,76.650002,19.171511,58.020000,42.779999,21.629999,NaN,108.540001,44.279999,44.799999,21.782499,NaN,22.145924,19.799999,15.05,28.719999,6.06,70.754967,35.0,34.459999,41.209999,37.000000,NaN,22.209999,43.049999,35.299999,28.440001,18.666000,21.049999,10.50,NaN,45.660000,NaN,NaN,19.010000,...,15.853333,32.180000,53.689999,33.590000,13.59,31.444866,77.650002,6.17,20.230000,25.230000,80.790001,67.849998,27.459999,31.459999,48.119999,23.690001,37.560001,23.665001,53.480000,11.17,48.630001,37.380001,34.540001,36.980000,NaN,63.930000,34.009998,28.261351,30.125000,39.889999,16.860001,18.625000,17.969999,60.790001,17.695000,39.610001,15.11,3.61500,239.610001,29.851370
2010-01-06,24.420000,13.27,54.150002,103.946663,38.299999,37.470001,76.419998,19.595024,59.779999,42.029999,22.379999,NaN,109.529999,44.959999,45.330002,21.490000,NaN,22.067240,20.090000,14.91,28.480000,6.13,71.894043,35.0,34.889999,40.889999,36.720001,NaN,22.219999,43.230000,36.400002,27.610001,19.444000,20.700001,10.14,NaN,45.980000,NaN,NaN,19.000000,...,16.783333,32.150002,53.570000,33.349998,13.65,31.553503,76.510002,6.13,20.410000,25.219999,80.300003,66.959999,28.719999,31.799999,49.570000,23.570000,37.915001,23.665001,54.820000,11.22,47.939999,37.639999,34.889999,36.900002,NaN,64.449997,34.000000,28.596518,30.264999,40.020000,17.100000,18.592501,17.790001,60.900002,18.344999,39.430000,15.38,3.69500,234.669998,29.916569
2010-01-07,24.530001,13.55,54.110001,103.556664,37.990002,37.490002,75.970001,19.475725,62.200001,41.500000,22.434999,NaN,107.290001,46.889999,45.750000,21.690001,NaN,22.038626,20.059999,14.74,28.330000,6.25,71.682121,35.0,34.680000,41.540001,36.939999,NaN,22.080000,42.830002,36.500000,27.299999,19.247999,20.790001,10.80,NaN,48.119999,NaN,NaN,19.610001,...,16.673334,31.690001,53.599998,33.270000,14.11,32.064095,74.410004,6.39,20.860001,25.240000,80.190002,68.389999,29.230000,31.930000,49.230000,23.400000,37.404999,22.770000,56.750000,10.84,48.630001,37.919998,35.189999,38.259998,NaN,63.990002,34.080002,29.226633,29.915001,39.700001,17.490000,18.510000,17.549999,61.160000,18.594999,39.360001,15.82,3.65125,237.250000,29.627834
2010-01-08,24.660000,13.33,53.900002,102.986664,37.779999,37.270000,76.120003,19.505550,61.599998,40.669998,22.270000,NaN,106.769997,46.009998,45.990002,21.750000,NaN,22.031473,20.110001,15.00,28.500000,6.34,71.602646,33.5,34.500000,39.910000,36.990002,NaN,22.139999,43.160000,35.900002,27.100000,19.662001,21.160000,10.60,NaN,47.509998,NaN,NaN,19.799999,...,16.443333,31.770000,53.330002,33.540001,13.93,32.140141,75.610001,6.50,20.580000,24.820000,78.730003,67.900002,29.209999,31.940001,49.930000,24.950001,37.634998,23.430000,57.419998,11.10,48.560001,37.889999,35.610001,37.970001,NaN,64.209999,34.240002,30.064554,30.450001,39.410000,16.920000,

In [ ]:
df.dropna(axis=0, how='all', inplace=True)

In [ ]:
df.shape

(2263, 480)

In [ ]:
df.dropna(axis=1, how='any', inplace=True)

In [ ]:
df.shape

(2263, 429)

In [ ]:
df.isna().sum().sum()

0

In [ ]:
df_returns = pd.DataFrame()

In [ ]:
for name in df.columns:
  df_returns[name] = np.log(df[name]).diff()

In [ ]:
df_returns.head()

,CSCO,UAL,TROW,ISRG,PRGO,TPR,DVN,MRO,BA,VRTX,GILD,EQIX,TIF,MDT,V,A,MO,SWKS,MCHP,CDNS,WLTW,EIX,BBY,WBA,AJG,DTE,C,T,CF,DISH,MGM,HUM,WU,APH,SYY,MSI,FCX,ADM,LH,STI,...,AEE,DLTR,HAS,WMT,NTAP,KIM,BAX,LMT,KEY,UNM,BMY,PSA,WYNN,RHI,EFX,NUE,PKG,NBL,CTSH,SWK,MU,TRV,L,AEP,CI,JNJ,WM,DOV,FTI,AGN,M,CRM,PGR,WAT,BWA,LRCX,NWL,UAA,BLK,PPL
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,-0.004465,0.083163,0.011151,-0.004512,-0.039429,0.012045,0.001044,0.000934,0.032227,-0.033559,-0.000924,-0.009354,0.006571,0.020294,-0.011525,-0.010922,0.000505,0.015400,-0.015890,-0.009852,-0.002244,-0.008956,0.025063,-0.008075,-0.005388,-0.011318,0.037522,-0.004911,0.016094,-0.006157,0.076161,-0.000438,-0.004199,-0.021118,-0.004280,-0.013898,0.005973,0.005387,-0.012252,0.013606,...,-0.003970,-0.012744,-0.011125,-0.010008,-0.017705,-0.005138,-0.006886,0.010356,0.037990,0.014439,-0.015730,0.008203,0.059041,0.012459,-0.000318,0.006881,0.012744,0.025618,0.011261,0.014503,0.029067,-0.023975,0.005365,-0.011514,0.016633,-0.011663,-0.004401,-0.004733,0.017074,-0.009978,-0.011793,-0.004286,-0.003333,-0.013723,0.046560,-0.006793,-0.005939,0.029476,0.004308,-0.013019
2010-01-06,-0.006531,-0.047102,-0.015757,0.014405,-0.012713,0.019402,-0.003005,0.021850,0.029883,-0.017687,0.034086,0.009080,0.015240,0.011761,-0.013519,-0.003559,0.014540,-0.009346,-0.008392,0.011485,0.015971,0.012401,-0.007795,-0.007596,0.000450,0.004172,0.030686,-0.029619,0.040835,-0.016767,-0.034887,0.006984,-0.000526,0.003772,0.002855,0.013898,0.039124,-0.002532,-0.016708,0.027608,...,-0.006895,0.057007,-0.000933,-0.002238,-0.007171,0.004405,0.003449,-0.014790,-0.006504,0.008858,-0.000396,-0.006084,-0.013204,0.044863,0.010749,0.029688,-0.005078,0.009407,0.000000,0.024747,0.004466,-0.014290,0.006931,0.010082,-0.002166,0.008101,-0.000294,0.011790,0.004637,0.003254,0.014134,-0.001746,-0.010067,0.001808,0.036075,-0.004555,0.017711,0.021889,-0.020832,0.002182
2010-01-07,0.004494,0.020881,-0.000739,-0.003759,-0.008127,0.000534,-0.005906,-0.006107,0.039684,-0.012690,0.002455,-0.020663,0.042031,0.009223,0.009264,-0.001298,-0.001494,-0.011467,-0.005281,0.019387,-0.002952,-0.006037,0.015771,0.005973,-0.006321,-0.009296,0.002743,-0.011291,-0.010131,0.004338,0.063058,0.045491,0.031601,-0.006667,0.001069,0.019876,-0.019313,-0.010511,-0.000270,0.057471,...,-0.009513,-0.006576,-0.014411,0.000560,-0.002402,0.033144,0.016052,-0.027831,0.041539,0.021809,0.000793,-0.001371,0.021131,0.017602,0.004080,-0.006883,-0.007239,-0.013542,-0.038553,0.034601,-0.034455,0.014290,0.007411,0.008562,0.036193,-0.007163,0.002350,0.021795,-0.011632,-0.008028,0.022551,-0.004447,-0.013583,0.004260,0.013536,-0.001777,0.028207,-0.011911,0.010934,-0.009698
2010-01-08,0.005286,-0.016369,-0.003889,-0.005519,-0.005543,-0.005886,0.001973,0.001530,-0.009693,-0.020203,-0.007382,-0.004859,-0.018946,0.005232,0.002762,-0.000325,0.002489,0.017485,0.005983,0.014297,-0.001109,-0.005204,-0.040030,0.001353,0.002714,0.007675,-0.016575,-0.007353,0.021281,0.017640,-0.018692,-0.012758,0.009642,0.007331,-0.006429,-0.046579,0.028320,-0.012567,-0.001484,0.019751,...,-0.007010,-0.013891,0.002521,-0.005050,0.008083,-0.012839,0.002369,0.015998,0.017068,-0.013514,-0.016780,-0.018375,-0.007191,-0.000684,0.000313,0.014119,0.064138,0.006130,0.028573,0.011737,0.023702,-0.001440,-0.000791,0.011865,-0.007609,0.003432,0.004684,0.028266,0.017726,-0.007332,-0.033133,0.001485,0.009075,0.000817,-0.018454,0.024841,-0.003166,-0.002056,0.007014,-0.003149


In [ ]:
df_returns['SPY'] = df_returns['SPY'].shift(-1)
df_returns['SPY'].tail()

Date
2018-12-21   -0.026778
2018-12-24    0.049290
2018-12-26    0.007648
2018-12-27   -0.001291
2018-12-28         NaN
Name: SPY, dtype: float64

In [ ]:
Ntest = 1000
train = df_returns.iloc[1:-Ntest]
test = df_returns.iloc[-Ntest:-1]

In [ ]:
df_returns.columns

Index(['CSCO', 'UAL', 'TROW', 'ISRG', 'PRGO', 'TPR', 'DVN', 'MRO', 'BA',
       'VRTX',
       ...
       'M', 'CRM', 'PGR', 'WAT', 'BWA', 'LRCX', 'NWL', 'UAA', 'BLK', 'PPL'],
      dtype='object', length=429)

In [ ]:
# x_cols = df.columns.drop('SPY')
x_cols = ['AAPL', 'MSFT', 'AMZN', 'JNJ', 'V', 'PG', 'JPM']
x_cols

['AAPL', 'MSFT', 'AMZN', 'JNJ', 'V', 'PG', 'JPM']

In [ ]:
Xtrain = train[x_cols]
Ytrain = train['SPY']
Xtest = test[x_cols]
Ytest = test['SPY']

In [ ]:
Xtrain.head()

,AAPL,MSFT,AMZN,JNJ,V,PG,JPM
Date,,,,,,,
2010-01-05,0.001727,0.000323,0.005883,-0.011663,-0.011525,0.000327,0.019185
2010-01-06,-0.016034,-0.006156,-0.018282,0.008101,-0.013519,-0.004755,0.005479
2010-01-07,-0.001850,-0.010454,-0.017160,-0.007163,0.009264,-0.005438,0.019615
2010-01-08,0.006626,0.006873,0.026717,0.003432,0.002762,-0.001323,-0.002459
2010-01-11,-0.008861,-0.012802,-0.024335,0.000156,-0.002878,-0.003979,-0.003363


In [ ]:
Ytrain.head()

Date
2010-01-05    0.000704
2010-01-06    0.004212
2010-01-07    0.003322
2010-01-08    0.001396
2010-01-11   -0.009370
Name: SPY, dtype: float64

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(Xtrain, Ytrain)
model.score(Xtrain, Ytrain), model.score(Xtest, Ytest)

(0.008271754178782342, -0.011369618185063102)

In [ ]:
# Direction
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
np.mean(np.sign(Ptrain) == np.sign(Ytrain)), np.mean(np.sign(Ptest) == np.sign(Ytest))

(0.528526148969889, 0.5285285285285285)

In [ ]:
set(np.sign(Ptrain)), set(np.sign(Ptest))

({-1.0, 1.0}, {-1.0, 1.0})

In [ ]:
train_idx = df.index <= train.index[-1]
test_idx = df.index > train.index[-1]

train_idx[0] = False
test_idx[-1] = False

In [ ]:
df_returns['Position'] = 0 # create new column
df_returns.loc[train_idx,'Position'] = (Ptrain > 0)
df_returns.loc[test_idx,'Position'] = (Ptest > 0)

In [ ]:
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']

In [ ]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

0.5477174926058384

In [ ]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.3037759916492

In [ ]:
# Total return buy-and-hold train
Ytrain.sum()

0.5863972019814705

In [ ]:
# Total return buy-and-hold test
Ytest.sum()

0.19307543946998518

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=10)
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(0.5586370839936609, 0.5285285285285285)

In [ ]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [ ]:
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']

In [ ]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

0.5905718759742236

In [ ]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.19680747922142316

In [ ]:
# Total return buy-and-hold
Ytrain.sum(), Ytest.sum()

(0.5863972019814705, 0.19307543946998518)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=2)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(1.0, 0.5255255255255256)

In [ ]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)

In [ ]:
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest

In [ ]:
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']

In [ ]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

4.706503557502184

In [ ]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.3839643540537523

In [ ]:
# Total return buy-and-hold
Ytrain.sum(), Ytest.sum()

(0.5863972019814705, 0.19307543946998518)